In [14]:
import pandas as pd
import gtfstk as gt
import geopandas as gpd
import datetime as dt
import numpy as np
import tkinter as tk
import requests, zipfile, io, os, re, shutil
from tkinter import filedialog
from shapely.geometry import Point
%matplotlib inline

In [15]:
crs={'init':'epsg:4326'} # WGS84 projection
buffer_distance=.125
one_deg_lat=69.05397727272727 # miles
one_deg_lon=48.99318181818182 # miles
conversion_deg=(np.mean([one_deg_lat,one_deg_lon]))
buffer_distance_deg=buffer_distance/conversion_deg

In [16]:
idaho='16'
ada_county='001'
canyon_county='027'
places_url=r"https://www2.census.gov/geo/tiger/TIGER2018/PLACE/tl_2018_16_place.zip"
counties_url=r"https://www2.census.gov/geo/tiger/TIGER2018/COUNTY/tl_2018_us_county.zip"

In [17]:
def readOnlineShapefile(url,name):
    zip_file_url=url
    r=requests.get(zip_file_url)
    z=zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall(str(name))
    gdf=gpd.read_file(str(name))
    shutil.rmtree(str(name))
    return gdf
places=readOnlineShapefile(places_url,'places')

In [18]:
cities=['Boise City',
        'Caldwell',
        'Eagle',
        'Garden City',
        'Greenleaf',
        'Kuna',
        'Melba',
        'Meridian',
        'Middleton',
        'Nampa',
        'Notus',
        'Parma',
        'Star',
        'Wilder']
places=places.loc[places['NAME'].isin(cities)]
places.sort_values('NAME',inplace=True)

In [ ]:
def getVRTServiceArea(url,name):
    gdf=readOnlineShapefile(url,name)
    gdf=gdf.loc[(gdf['STATEFP']==idaho)&((gdf['COUNTYFP']==ada_county)|(gdf['COUNTYFP']==canyon_county))]
    return gdf
counties=getVRTServiceArea(counties_url,'counties')

In [ ]:
unincorporated=gpd.overlay(counties,places,how='difference')
jurisdictions=gpd.GeoDataFrame(pd.concat([places,unincorporated],ignore_index=True))

In [ ]:
# jurisdictions['NAMELSAD']=jurisdictions['NAMELSAD'].str.title()
jurisdictions['NAMELSAD'].replace({'city':''},regex=True,inplace=True)

cityNamesArray=jurisdictions.NAMELSAD.unique()
cityNames={i:cityNamesArray[i] for i in range(len(jurisdictions.NAMELSAD.unique()))}
cityNames[len(cityNames)+1]='All'
cityNames=cityNames.values()

In [ ]:
def button_stop_summary_clicked():
    root.stop_summary=filedialog.askopenfilename(initialdir="/",title="Select Stop Summary Report",filetypes = (("Excel","*.xlsx"),("all files","*.*")))
def button_outfolder_clicked():
    root.outfolder=filedialog.askdirectory(initialdir="/",title="Select Output folder")
def button_cities_choices_clicked(*args):
    root.city_choice=tkvar.get()
def close_window():
    root.city_choice=tkvar.get()
    root.destroy()

root=tk.Tk()
root.title('Stops Cities')
# root.geometry('350x200')

tkvar=tk.StringVar(root)
tkvar.set("All")

label_stop_summary=tk.Label(root,text='Select Stop Summary Report')
label_stop_summary.grid(column=0,row=0)

button_stop_summary=tk.Button(root,text="Browse",command=button_stop_summary_clicked)
button_stop_summary.grid(column=0,row=1)

label_out=tk.Label(root,text="Select an output folder")
label_out.grid(column=0,row=2)

button_out=tk.Button(root,text="Browse",command=button_outfolder_clicked)
button_out.grid(column=0,row=3)

city_label=tk.Label(root,text="Pick a City")
city_label.grid(column=0,row=4)

cities_choices=tk.OptionMenu(root,tkvar,*cityNames)
cities_choices.grid(column=0,row=5)

button_run=tk.Button(root,text='Run',width=25,command=close_window)
button_run.grid(column=0,row=6)
root.mainloop()

In [13]:
root.city_choice

'Garden  '

In [10]:
outfolder=root.outfolder
if os.path.exists(outfolder):
    outfolder=outfolder+"\{}.{}"
else:
    os.mkdir(root.outfolder)
    outfolder=outfolder+"\{}.{}"
filename=os.path.basename(root.stop_summary)
filename=os.path.splitext(filename)[0]
filename=root.city_choice+"_"+filename

AttributeError: '_tkinter.tkapp' object has no attribute 'outfolder'

In [ ]:
stop_sum=pd.read_excel(root.stop_summary,dtype={'TRIP_TIME':str,'STOP_ID':str,'SCHEDULED_TIME':str})
stop_sum['ROUTE']=stop_sum['ROUTE'].astype(str)
stop_sum['ROUTE'].replace({'71':'7A','72':'7B','80':'8x'},inplace=True)

In [ ]:
stop_sum['geometry']=[Point(xy) for xy in zip(stop_sum.GTFS_LONG,stop_sum.GTFS_LAT)]
stop_sum=gpd.GeoDataFrame(stop_sum,geometry='geometry',crs=crs)

In [ ]:
stop_sum_cities=gpd.sjoin(stop_sum,jurisdictions,how='left')

In [ ]:
if root.city_choice!='All':
    city_stops=stop_sum_cities.loc[stop_sum_cities['NAMELSAD']==root.city_choice]
    city_stops=city_stops[['ROUTE','STOP_ID','STOPNAME','ON','OFF','geometry']]
else:
    city_stops=stop_sum_cities.copy()
    city_stops=city_stops[['ROUTE','STOP_ID','STOPNAME','ON','OFF','geometry']]

In [ ]:
city_stops_on=city_stops.copy()
city_stops_on['ROUTE']=city_stops_on['ROUTE'].astype(str)+"_on"
city_stops_on=city_stops_on.pivot_table(index=['STOP_ID'],columns='ROUTE',values='ON',aggfunc='sum')
city_stops_on.reset_index(inplace=True)
city_stops_on=city_stops_on.merge(city_stops,how='left')
city_stops_on=gpd.GeoDataFrame(city_stops_on,geometry='geometry',crs=crs)
city_stops_on=city_stops_on.dissolve(['STOP_ID','STOPNAME'],aggfunc='mean',as_index=False)
city_stops_on.drop(['ON','OFF','geometry'],axis=1,inplace=True)

In [ ]:
city_stops_off=city_stops.copy()
city_stops_off['ROUTE']=city_stops_off['ROUTE'].astype(str)+"_off"
city_stops_off=city_stops_off.pivot_table(index=['STOP_ID'],columns='ROUTE',values='OFF',aggfunc='sum')
city_stops_off.reset_index(inplace=True)
city_stops_off=city_stops_off.merge(city_stops,how='left')
city_stops_off=gpd.GeoDataFrame(city_stops_off,geometry='geometry',crs=crs)
city_stops_off=city_stops_off.dissolve(['STOP_ID','STOPNAME'],aggfunc='mean',as_index=False)
city_stops_off.drop(['ON','OFF','geometry'],axis=1,inplace=True)
# city_stops_off=gpd.GeoDataFrame(city_stops_off,geometry='geometry',crs=crs)

In [ ]:
city_stops2=city_stops_on.merge(city_stops_off)
city_stops2=city_stops2.merge(city_stops,how='left',left_on='STOP_ID',right_on='STOP_ID')
city_stops2=gpd.GeoDataFrame(city_stops2,geometry='geometry',crs=crs)
city_stops2.drop(['STOPNAME_y','ROUTE','ON','OFF'],axis=1,inplace=True)
city_stops2.rename(columns={'STOPNAME_x':'STOPNAME'},inplace=True)
city_stops2=city_stops2.dissolve(by='STOP_ID',aggfunc='first',as_index=False)
city_stops2=city_stops2.loc[city_stops2.geometry.geom_type=='Point']

In [ ]:
city_stops2.to_file(outfolder.format((filename),".shp"))

In [ ]:
city_stops2.drop('geometry',axis=1,inplace=True)
city_stops2.to_excel(outfolder.format((filename),".xlsx"),index=False)